In [1]:
import numpy as np
import TensorFrost as tf

tf.initialize(tf.opengl)

# Householder reflection
def householder_reflection(a):
    v = a.copy()
    v[0] = v[0] + np.copysign(np.linalg.norm(v), a[0])
    v = v / np.linalg.norm(v)
    return np.eye(len(a)) - 2 * np.outer(v, v)

# QR decomposition using Householder reflections
def qr_decomposition(A):
    m, n = A.shape
    Q = np.eye(m)
    R = A.copy()
    for i in range(min(m, n)):
        H = np.eye(m)
        H[i:, i:] = householder_reflection(R[i:, i])
        Q = Q @ H
        R = H @ R
    return Q, R


def modified_gram_schmidt(A):
    """
    Implements the Modified Gram-Schmidt orthogonalization to get the QR decomposition of matrix A.
    A = QR
    """
    A = A.astype(float)  # Ensure A is of float type
    m, n = A.shape
    Q = np.zeros((m, n))
    R = np.zeros((n, n))
    
    for i in range(n-1):
        R[i, i] = np.linalg.norm(A[:, i])
        Q[:, i] = A[:, i] / R[i, i]
        R[i, i+1:n] = np.dot(Q[:, i].T, A[:, i+1:n])
        A[:, i+1:n] -= np.outer(Q[:, i], R[i, i+1:n])
    R[n-1, n-1] = np.linalg.norm(A[:, n-1])
    Q[:, n-1] = A[:, n-1] / R[n-1, n-1]
    return Q, R

#dynamic size QR decomposition
def QRDecomposition():
    A = tf.input([-1, -1], tf.float32)

    m, n = A.shape
    Q = tf.zeros([m, n])
    R = tf.zeros([n, n])
    j = tf.index(0, [m])

    def loop_body(i):
        R[i, i] = tf.norm(A[j, i])
        Q[j, i] = A[j, i] / R[i, i]

        p, k = tf.index_grid([0, i + 1], [m, n])
        t, = tf.index_grid([i+1], [n])
        R[i, t] = tf.sum(Q[p, i] * A[p, k], axis=0)
        A[p, k] -= Q[p, i] * R[i, k]

    tf.loop(loop_body, 0, n-1, 1)

    R[n-1, n-1] = tf.norm(A[j, n-1])
    Q[j, n-1] = A[j, n-1] / R[n-1, n-1]

    return [Q, R]

qr = tf.compile(QRDecomposition)


TensorFrost module loaded!
QRDecomposition:
  Kernel count: 8
  Intermediate buffers: 0
  Host readbacks: 0
  Host writes: 0
  Lines of generated code: 429
  IR Compile time: 2.897100 ms
  Compiler time: 1634.011353 ms



In [2]:
#print(qr.list_operations())

In [3]:
#generate random matrix
A = np.random.rand(5, 5)

#compute QR decomposition
Q, R = modified_gram_schmidt(A)

#compute QR decomposition using TensorFrost
Atf = tf.tensor(A)
Qtf, Rtf = qr(Atf)
Qnp = Qtf.numpy
Rnp = Rtf.numpy

#check if QR decomposition is correct
print("QR decomposition is correct:", np.allclose(A, np.dot(Q, R)))
print("QR decomposition using TensorFrost is correct:", np.allclose(A, np.dot(Qnp, Rnp)))

#check error
print("Error:", np.linalg.norm(A - np.dot(Q, R)))
print("Error using TensorFrost:", np.linalg.norm(A - np.dot(Qnp, Rnp)))

#print Q and R
print("Q:\n", Qnp)
print("R:\n", Rnp)


QR decomposition is correct: True
QR decomposition using TensorFrost is correct: True
Error: 2.2030305172320775e-16
Error using TensorFrost: 1.0693669784371055e-07
Q:
 [[ 0.18342032  0.7535508  -0.10644833 -0.02849421 -0.6215908 ]
 [ 0.6835706  -0.36019298 -0.2872629   0.52576804 -0.2098579 ]
 [ 0.01957198  0.507313    0.15320423  0.6315658   0.5656001 ]
 [ 0.31524605 -0.08165301  0.93082225 -0.02281861 -0.16432247]
 [ 0.6319218   0.19592991 -0.12746401 -0.5686475   0.4718888 ]]
R:
 [[ 1.1499681   0.8690763   0.88347316  1.0972072   0.85174567]
 [ 0.          1.1123344   0.67282724  1.0126327   0.22717237]
 [ 0.          0.          0.79170537  0.41513917  0.1916663 ]
 [ 0.          0.          0.          0.6089157  -0.16332015]
 [ 0.          0.          0.          0.          0.25340018]]


In [4]:
#performance test
import time
A = np.random.rand(5000, 5000).astype(np.float32)

#naive NumPy QR decomposition
#start = time.time()
#Q, R = modified_gram_schmidt(A)
#print("Time for naive NumPy QR decomposition:", time.time() - start)

#TensorFrost QR decomposition
Atf = tf.tensor(A)
start = time.time()
Qtf, Rtf = qr(Atf)
print("Time for TensorFrost QR decomposition:", time.time() - start)

#householder QR decomposition
#start = time.time()
#Q, R = qr_decomposition(A)
#print("Time for householder QR decomposition:", time.time() - start)

#built-in NumPy QR decomposition
start = time.time()
Q, R = np.linalg.qr(A)
print("Time for built-in NumPy QR decomposition:", time.time() - start)

print("Error:", np.linalg.norm(A - np.dot(Q, R)))
print("Error using TensorFrost:", np.linalg.norm(A - np.dot(Qtf.numpy, Rtf.numpy)))

Time for TensorFrost QR decomposition: 0.16500163078308105
Time for built-in NumPy QR decomposition: 4.173810720443726
Error: 0.0012286535
Error using TensorFrost: 0.0018777299
